
# Preparação e Análise de Dados para Redes Neurais


## Objetivo

Explorar separabilidade de classes em 2D, projetar dados 5D para 2D com PCA e preparar o dataset **Spaceship Titanic** para redes neurais com ativação `tanh`.


## Exercício 1 — Dados 2D (4 classes)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

N = 100
params = {
    0: {"mean": [2, 3],  "std": [0.8, 2.5]},
    1: {"mean": [5, 6],  "std": [1.2, 1.9]},
    2: {"mean": [8, 1],  "std": [0.9, 0.9]},
    3: {"mean": [15, 4], "std": [0.5, 2.0]},
}

Xs, ys = [], []
for c, p in params.items():
    mean = np.array(p["mean"])
    std = np.array(p["std"])
    Xc = np.random.randn(N, 2) * std + mean
    Xs.append(Xc)
    ys.append(np.full(N, c))

X = np.vstack(Xs)
y = np.hstack(ys)

plt.figure(figsize=(7,5))
for c in params.keys():
    plt.scatter(X[y==c,0], X[y==c,1], s=12, label=f"Classe {c}", alpha=0.8)
plt.xlabel("x1"); plt.ylabel("x2")
plt.title("Exercício 1 — Distribuição 2D (4 classes)")
plt.legend()
plt.show()


As quatro classes se distribuem em regiões distintas do plano, cada uma concentrada em torno de um centro específico. Embora o eixo *x1* contribua fortemente para a separação, a variabilidade em *x2* cria dispersões diferentes entre os grupos. Esse cenário exige a combinação de múltiplas fronteiras de decisão ou o uso de modelos não lineares para capturar de forma adequada a estrutura dos dados.

In [ ]:

# Superfícies de decisão com MLP
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(16,), activation="tanh", max_iter=2000, random_state=42)
clf.fit(X, y)

xx, yy = np.meshgrid(
    np.linspace(X[:,0].min()-1, X[:,0].max()+1, 300),
    np.linspace(X[:,1].min()-1, X[:,1].max()+1, 300),
)
ZZ = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.figure(figsize=(7,5))
plt.contourf(xx, yy, ZZ, alpha=0.25, levels=[-0.5,0.5,1.5,2.5,3.5])
for c in params.keys():
    plt.scatter(X[y==c,0], X[y==c,1], s=10, label=f"Classe {c}")
plt.xlabel("x1"); plt.ylabel("x2")
plt.title("Exercício 1 — Superfícies de decisão (MLP tanh)")
plt.legend()
plt.show()


## Exercício 2 — Dados 5D (A/B) + PCA(2D)

In [ ]:

from sklearn.decomposition import PCA

np.random.seed(7)

muA = np.array([0, 0, 0, 0, 0])
SigmaA = np.array([
    [1.0, 0.8, 0.1, 0.0, 0.0],
    [0.8, 1.0, 0.3, 0.0, 0.0],
    [0.1, 0.3, 1.0, 0.5, 0.0],
    [0.0, 0.0, 0.5, 1.0, 0.2],
    [0.0, 0.0, 0.0, 0.2, 1.0]
])

muB = np.array([1.5, 1.5, 1.5, 1.5, 1.5])
SigmaB = np.array([
    [1.5, -0.7, 0.2, 0.0, 0.0],
    [-0.7, 1.5, 0.4, 0.0, 0.0],
    [0.2, 0.4, 1.5, 0.6, 0.0],
    [0.0, 0.0, 0.6, 1.5, 0.3],
    [0.0, 0.0, 0.0, 0.3, 1.5]
])

XA = np.random.multivariate_normal(muA, SigmaA, size=500)
XB = np.random.multivariate_normal(muB, SigmaB, size=500)
X5 = np.vstack([XA, XB])
y5 = np.hstack([np.zeros(500), np.ones(500)])

pca = PCA(n_components=2, random_state=42)
X2 = pca.fit_transform(X5)

plt.figure(figsize=(7,5))
plt.scatter(X2[y5==0,0], X2[y5==0,1], s=10, label="Classe A")
plt.scatter(X2[y5==1,0], X2[y5==1,1], s=10, label="Classe B")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.title("Exercício 2 — PCA (5D → 2D)")
plt.legend()
plt.show()


A projeção via PCA mostra que as classes A e B têm centros deslocados, mas ainda apresentam forte sobreposição devido à variância dentro de cada grupo. Essa configuração torna a separação linear pouco eficaz, já que não existe um hiperplano simples que separe bem as duas classes. Modelos mais expressivos, que incorporam não-linearidades, são mais adequados para capturar as fronteiras complexas observadas no espaço reduzido.

## Exercício 3 — Spaceship Titanic: pré-processamento

In [ ]:
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
plt.rcParams["figure.figsize"] = (8,3.5)

In [ ]:
csv_path = "data/train.csv"
assert os.path.exists(csv_path), f"Arquivo não encontrado: {csv_path}"

df = pd.read_csv(csv_path)

target_col = "Transported"
num_cols = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
cat_cols = ["HomePlanet", "CryoSleep", "Destination", "VIP", "Cabin"]

# Colunas de gastos com forte assimetria (usar log1p)
spend_cols = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]


#### Objetivo: 

Prever `Transported`, indicando se o passageiro foi transportado para outra dimensão.  

#### Numéricas:

`Age`, `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck`.  

#### Categóricas:

`HomePlanet`, `CryoSleep`, `Destination`, `VIP`, `Cabin`.


In [ ]:
print("Tipos detectados:")
display(df[num_cols + cat_cols + [target_col]].dtypes)

na_count = df[num_cols + cat_cols + [target_col]].isna().sum().sort_values(ascending=False)
na_pct = (na_count/len(df)).round(3)
missing_report = pd.DataFrame({"missing": na_count, "pct": na_pct})
display(missing_report[missing_report["missing"] > 0])

In [ ]:
# Transported vem como bool -> int (0/1)
if df[target_col].dtype != int:
    df[target_col] = df[target_col].astype(int)

X = df[num_cols + cat_cols].copy()
y = df[target_col].values

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("Train:", X_train.shape, "Val:", X_val.shape)


In [ ]:
def plot_before_after(series, transformer, title_after):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3.5))
    ax1.hist(series.dropna().values, bins=40)
    ax1.set_title(f"{series.name} — original")
    tr = transformer.fit_transform(series.to_frame())
    ax2.hist(tr.ravel(), bins=40)
    ax2.set_title(f"{series.name} — {title_after}")
    plt.tight_layout(); plt.show()

age_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
plot_before_after(df["Age"], age_pipe, "padronizada")


In [ ]:
food_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("log", FunctionTransformer(np.log1p, validate=False)),
    ("scaler", StandardScaler())
])
plot_before_after(df["FoodCourt"], food_pipe, "log1p + padronizada")


O dataset **Spaceship Titanic** busca prever a variável *Transported*, que indica se um passageiro foi levado para outra dimensão. Ele combina atributos numéricos (`Age`, `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck`) e categóricos (`HomePlanet`, `CryoSleep`, `Destination`, `VIP`, `Cabin`). A análise inicial mostrou valores ausentes em quase todas as colunas, em torno de 2%, tratados com a mediana para numéricos, robusta a outliers, e o valor mais frequente para categóricos, preservando consistência.

Na preparação, as variáveis categóricas foram codificadas via one-hot encoding, enquanto os atributos numéricos seguiram duas estratégias: `Age` foi apenas padronizada para média zero e desvio um, e as variáveis de gastos passaram antes por uma transformação logarítmica (`log1p`) para reduzir caudas longas, sendo então padronizadas. Essa combinação é apropriada para redes neurais com função `tanh`, já que estabiliza escalas, centraliza valores e melhora a propagação do gradiente.

Os histogramas de `Age` e `FoodCourt` ilustram esses efeitos: no primeiro caso, a distribuição foi recentrada e normalizada, enquanto no segundo a transformação logarítmica tornou a escala mais equilibrada antes da padronização. O resultado final é um conjunto de dados mais uniforme e adequado ao treinamento, reduzindo distorções e favorecendo desempenho do modelo.

## Conclusões

Os experimentos mostraram que, em dados sintéticos 2D, a separação linear não é suficiente, exigindo funções de ativação não lineares para capturar fronteiras mais complexas. Na projeção dos dados 5D em 2D, a sobreposição causada por correlações entre atributos reforça essa limitação e destaca a necessidade de modelos mais expressivos. Já no caso do Spaceship Titanic, o pré-processamento com imputação, codificação categórica e padronização numérica foi fundamental para tornar o conjunto compatível com redes neurais baseadas em `tanh`, garantindo maior estabilidade no treinamento e melhor capacidade de generalização.
